# **Ejercicios de pair programming Módulo 3 Sprint 1**
## **Regresión lineal: Lección 11**
### **Decision Tree**

---

In [1]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

pd.options.display.max_columns = None

En el pair programming de hoy debéis usar el csv que guardastéis cuando hicistéis el pairprgramming de codificicación (este csv debería tener las variables estadandarizas).

In [2]:
df_socio = pd.read_csv('../datos/esperanza_vida_encoding_lidya.csv', index_col=0)
df_socio.head(2)

,region,population_in_millions,population_density,urban_population_y,gdp_per_capita,health_total_expenditure,health_care_index,health_physicians,quantity_of_physicians,quality_of_life_index,purchasing_power_index,safety_index,pop_using_improved_drinking_water_urban,pop_using_improved_drinking_water_rural,pollution_index,life_expectancy_at_birth_total_bc
country,,,,,,,,,,,,,,,,
Argentina,4,44.271,16.2,91.749,14564.5,4.8,73.30,3.8,3.0,139.59,58.40,37.37,96.2,98.3,53.33,1.596496e+11
Australia,5,24.451,3.2,85.904,51352.2,9.4,74.25,3.4,2.5,176.54,101.94,57.58,100.0,100.0,24.58,2.611737e+11



En pairprogramming anteriores ajustastéis vuestro datos a una regresión lineal. El objetivo de hoy es:
 - Ajustar el modelo a un Decision Tree


In [3]:
X = df_socio.drop('life_expectancy_at_birth_total_bc', axis=1)
y = df_socio['life_expectancy_at_birth_total_bc']

In [4]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 63)

arbol = DecisionTreeRegressor(random_state = 137)

arbol.fit(x_train, y_train)

max_features = np.sqrt(len(x_train.columns))
print(f'La máxima cantidad de features que podemos incluir en el modelo es {max_features}.')
print(f'La máxima profundidad que puede tener nuestro modelo es {arbol.tree_.max_depth}')

y_pred_test = arbol.predict(x_test)
y_pred_train = arbol.predict(x_train)

La máxima cantidad de features que podemos incluir en el modelo es 3.872983346207417.
La máxima profundidad que puede tener nuestro modelo es 10


In [5]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                "set": ["test", "train"]}
    
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [6]:
resultados1 = metricas(y_test, y_train, y_pred_test, y_pred_train, 'Arbol de decisión Lidya 1')

In [7]:
resultados1

,MAE,MSE,RMSE,R2,set,modelo
0,4.192010e+10,3.330493e+21,5.771042e+10,-0.029296,test,Arbol de decisión Lidya 1
1,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,train,Arbol de decisión Lidya 1


>Vemos que tenemos un claro caso de overfitting, así que utilizamos GridSearchCV para ajustar nuestro hiperparámetros.

In [8]:
print(f'La máxima cantidad de features que podemos incluir en el modelo es {max_features}.')
print(f'La máxima profundidad que puede tener nuestro modelo es {arbol.tree_.max_depth}')

La máxima cantidad de features que podemos incluir en el modelo es 3.872983346207417.
La máxima profundidad que puede tener nuestro modelo es 10


In [9]:
param = {"max_depth": [2,3,4,6],  
        "max_features": [3,4,5,6],        
        "min_samples_split": [10,50,100],
        "min_samples_leaf": [10,50,100]}

In [10]:
gs = GridSearchCV(
            estimator=DecisionTreeRegressor(), 
            param_grid= param, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error")

In [11]:
gs.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [2, 3, 4, 6],
                         'max_features': [3, 4, 5, 6],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [10, 50, 100]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [12]:
modelo1 = gs.best_estimator_
modelo1

DecisionTreeRegressor(max_depth=2, max_features=3, min_samples_leaf=10,
                      min_samples_split=10)

In [13]:
param2 = {"max_depth": [2,3,4,6],  
        "max_features": [3,4,5,6],    
        "min_samples_split": [2,4,5,10,15],
        "min_samples_leaf": [2,4,5,10,15]}

In [14]:
gs2 = GridSearchCV(
            estimator=DecisionTreeRegressor(), 
            param_grid= param2, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error")

gs2.fit(x_train, y_train)

modelo2 = gs2.best_estimator_
modelo2

DecisionTreeRegressor(max_depth=4, max_features=4, min_samples_leaf=2,
                      min_samples_split=10)

In [15]:
#volvemos a hacer el método con los hiperparámetros que nos ha devuelto el GridSearchCV

x_train2, x_test2, y_train2, y_test2 = train_test_split(X, y, test_size = 0.3, random_state = 63)

arbol2 = DecisionTreeRegressor(max_depth=3, max_features=5, min_samples_leaf=4,
                      min_samples_split=10)

arbol2.fit(x_train2, y_train2)

y_pred_test2 = arbol2.predict(x_test2)
y_pred_train2 = arbol2.predict(x_train2)

resultados2 = metricas(y_test2, y_train2, y_pred_test2, y_pred_train2, 'Arbol de decisión Lidya 2')
resultados2

,MAE,MSE,RMSE,R2,set,modelo
0,3.015948e+10,1.361269e+21,3.689538e+10,0.579297,test,Arbol de decisión Lidya 2
1,1.580369e+10,3.960794e+20,1.990174e+10,0.894034,train,Arbol de decisión Lidya 2


 - Extraer las métricas


In [16]:
resultados = pd.read_csv("../datos/esperanza_vida_metricas_lidya.csv", index_col =0)

In [18]:
resultados = pd.concat([resultados, resultados1, resultados2], axis = 0)

In [19]:
resultados

,MAE,MSE,RMSE,R2,set,modelo
0,3.187947e+10,1.516556e+21,3.894299e+10,0.393487,test,Regresión Lineal
1,1.664577e+10,4.360158e+20,2.088099e+10,0.892042,train,Regresión Lineal
0,4.192010e+10,3.330493e+21,5.771042e+10,-0.029296,test,Arbol de decisión Lidya 1
1,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,train,Arbol de decisión Lidya 1
0,4.823715e+10,3.636189e+21,6.030082e+10,-0.123773,test,Arbol de decisión Lidya 2
1,2.598880e+10,1.104433e+21,3.323301e+10,0.704522,train,Arbol de decisión Lidya 2
0,4.192010e+10,3.330493e+21,5.771042e+10,-0.029296,test,Arbol de decisión Lidya 1
1,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,train,Arbol de decisión Lidya 1
0,3.015948e+10,1.361269e+21,3.689538e+10,0.579297,test,Arbol de decisión Lidya 2
1,1.580369e+10,3.960794e+20,1.990174e+10,0.894034,train,Arbol de decisión Lidya 2


 - Debatid entre vosotras que modelo es mejor y por qué (basándose en las métricas)

>Ninguno de los modelos que hemos intentando hacer hasta ahora nos ha dado un resultado satisfactorio.
> El mejor de ellos es el segundo árbol de decisión, y aún así éste sufre un enorme overfitting.

In [20]:
resultados.to_csv("../datos/esperanza_vida_metricas_lidya.csv")